# Project Introduction:
Primer laboratorio de aprendizaje automático, desarrollado por Sergio Barragán Blanco (100472343) y Eduardo Alarcón Navarro (100472175). 
Grupo 17.


# 1. EDA
Existen 22 carácterísticas que definen cada momento, de las cuales ninguna es categórica, todas son numéricas (con la energía suman 23). No existen valores faltantes, pero si los hubiera, los rellenaríamos con la media del valor superior e inferior.

No existen tampoco columnas constantes, que se eliminarían. 

Con todo esto, podemos observar que es un problema de regresión.

La variable que estamos intentando predecir es la "energía" que es el valor de la energía generada 24 horas después. 

Por lo tanto, vamos a comenzar con todos los imports necesarios para este proyecto

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import time
from sklearn.dummy import DummyRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, Normalizer, PowerTransformer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, TimeSeriesSplit, cross_val_score
from sklearn.tree import DecisionTreeRegressor  
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from joblib import dump, load






Posteriormente cargamos el dataset de wind_ava.csv, y eliminamos todas las columnas a excepción de la energía y los datos que usaremos para predecirla (13)

In [ ]:

# Load the data
data = pd.read_csv('wind_ava.csv.gz', compression='gzip')

# FIlter the data to only include the columns that end in 13
data = data.filter(regex='13$|energy')
#print(data)
print(data.head())
print("--------------------")



# Correlación:
## Correlaciones entre parámetros:
Al principio, parece que las columnas lai_lv.13 y lai_hv.13 tienen una correlación, pero según avanza el tiempo, desaparece.

## Escala de los datos
Entre las diferentes columnas de datos, tenemos valores y magnitudes muy dispares.

Para comprobar la posible correlación entre datos, hemos usado la librería de matplotlib para hacer una matriz de correlación:

In [ ]:
# Step 1: Load Data, done in the first cell

# Step 2: Calculate Correlation
correlation_matrix = data.corr()

# Step 3: Visualize Correlation Matrix
plt.figure(figsize=(15, 12))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix with all columns')
plt.show()


# Step 4: Filter out columns with correlation more than 0.9
# Al eliminar las columnas innecesarias, el modelo empeora en la predicción de la energía considerablemente.
data_filtered = data.drop(columns=['lai_hv.13', 'u10.13', 'v10.13', 'stl3.13',
                                    'iews.13', 'inss.13', 'u100.13', 'v100.13',
                                      't2m.13', 'stl1.13', 'stl2.13'])
print(data_filtered.head())

# Repeat Step 3: Visualize Correlation Matrix
correlation_matrix = data_filtered.corr()
plt.figure(figsize=(15, 12))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix with selected columns')
plt.show()



A pesar de que la matriz de correlación sea mucho mejor, no se pueden sacar conclusiones claras de ella. Sobre todo, porque al entrenar los modelos, vemos que son peores.

Depues de realizar las pruebas con los dos datasets, hemos visto que es mejor utilizar todos los datos,ya que los resultados so mucho mejores. Por lo que vamos a proceder a separar los datos de entrenamiento de los datos de entrenamiento y los datos de test, con train_test_split de la fuente de datos original.

In [ ]:
y = data['energy']
X = data.drop(columns=['energy'])
print(X.head())

# test_size vamos a escoger 0.2 ya que tenemos datos en un rango de 5 años en orden (2015-2019) por lo que (asumiendo que el numero de datos para todos los años es similar) usaremos 2019 como test final.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
#print(f"Fechas de tain {train.iloc[0].datetime}-{train-iloc[-1].datetime}")

graph_data = {}

# Comportamiento futuro
Para este ejercicio vamos a llevar a cabo evaluaciones inner de varios tipos de modelos, los cuales poseerán distintos hyperparámetros que iremos ajustando. Posteriormente, compararemos los modelos y haremos una gráfica para representar visualmente las diferencias, y al mejor modelo lo seleccionaremos para probarlo con el conjunto de datos de test. Por último, entrenaremos ese modelo nuevamente, pero en este caso con todos los datos (incluidos el test) para poder mejorar (aunque sea ligeramente), la predicción a futuro del wind_comp.csv

En cuanto a las métricas que se van a usar, la principal será la Root Mean Squeared Error (RMSE), ya que es una forma compacta de evaluar los modelos, además penaliza gravemente los fallos, y es mas legible que el Mean Squared Error (MSE)

# Decidir usando KNN - Scalers

A continuación, usaremos KNN para determinar el método de escalado más apropiado para el problema. 
Dado que no queremos entrenar con datos en el futuro, no podremos usar una validacin cruzada normal (usando Kfold o Shuffle), por lo que tendremos que usar predefined split o Time Series split (el cual es igual a una validación cruzada, pero ignora los posteriores a test), en nuestro caso, usaremos TimeSeriesSplit para todas las validaciones. En cuanto a los scalers, probaremos "StandardScaler", "MinMaxScaler", "RobustScaler", "Normalizer" y "PowerTransformer".


In [ ]:
# First of all, the necessary scalers
scalers = [StandardScaler(), MinMaxScaler(), RobustScaler(), Normalizer(), PowerTransformer()]

# And we are going to store all the pipelines in order to test them later
pipelines = {}
for scaler in scalers:
    pipe = Pipeline([
        ('scaler', scaler),
        ('knn', KNeighborsRegressor())
    ])
    pipelines[str(scaler)[:-2]] = pipe

scores = {}
# Calculate the score for each scaler
for name, pipe in pipelines.items():
    scores[name] = -cross_val_score(pipe, X_train, y_train, cv=TimeSeriesSplit(), scoring='neg_root_mean_squared_error').mean()

# Print them all and compare which one is the best
for name, score in scores.items():
    print (f" {name}: {score}")

# Resultado con data sin filtrar
# StandardScaler: 455.56751873979965
# MinMaxScaler: 490.81263982750016
# RobustScaler: 454.0893589365546
# Normalizer: 644.0650360014222
# PowerTransformer: 428.29574620911944

# Resultado con data filtrada
# StandardScaler: 518.1803408987827
# MinMaxScaler: 538.7092117532035
# RobustScaler: 525.9490003734232
# Normalizer: 644.0650360014222
# PowerTransformer: 497.1992872267256

# Entrenamiento de diferentes modelos
Lo próximo que haremos, será probar con diferentes métodos de entrenamiento (KNN, árboles de regresión, regresión lineal normal y la variante de Lasso, y SVM. Para elegir el mejor, usaremos la librería time para saber cuánto se tarda en entrenar cada modelo, un dummy para comparar resultados, y el mejor de entre ellos, usando una política de importancia de 20% a tiempo y 80% precisión.

Lo primero que vamos a usar es el dummyRegressor, para tener una representación visual del peor escenario posible y que tanto mejoran nuestros diseños.

Como el scaler que dió mejor scoring fué el PowerTransformer, será el que usaremos de ahora en adelante

In [ ]:
# Create a Dummy regressor to obtain a baseline
scaler = PowerTransformer()

dummy = DummyRegressor(strategy='mean')
# We will use the previously mentioned scaler
pipe = Pipeline([
    ('scaler', scaler),
    ('model', dummy)
])
# Store the time so we can use it to compare every model
a = time.time()
# Train the model
pipe.fit(X_train, y_train)
b = time.time()
# We will use the training test to have a reference for the other models
y_pred = pipe.predict(X_test)

# But the later models will only be using the score with RMSE
mse_dummy = metrics.mean_squared_error(y_test, y_pred)
rmse_dummy = np.sqrt(metrics.mean_squared_error(y_test, y_pred))

print(f"MSE: {mse_dummy}")
print(f"RMSE: {rmse_dummy}")
print(f"Time taken to train the Dummy Model: {b-a} seconds")

graph_data["Dummy"] = [rmse_dummy, b-a]
# Resultado con data sin filtrar
# MSE: 439516.0127766374
# RMSE: 662.9600385970766
# Time taken to train the Dummy Model: 0.0872499942779541

# Resultado con data filtrada
# MSE: 439516.0127766374
# RMSE: 662.9600385970766
# Time taken to train the Dummy Model: 0.02886199951171875 seconds


Para empezar, vamos a comparar con un **Decision Tree Regresor**. En vez de usar un Grid Search, usamos un RandomSearch porque es más rápido y para probar más combinaciones de hiperparámetros.

In [ ]:
# DecisionTreeRegressor
# First of all is defining the parameters that will be used to train the model
param_grid = {
    'criterion': ['friedman_mse', 'absolute_error', 'poisson', 'squared_error'],  # Impurity criteria for regression
    'max_depth': [None, 10, 12, 13, 14, 15, 16, 17],  # Maximum depth of the tree
    'min_samples_split': range(2, 15, 2),  # Minimum number of samples to split an internal node
    'min_samples_leaf': range(10, 20, 2),  # Minimum number of samples in a leaf node
}
# Then, since we are doing a inner evaluation, we will use TimeSeriesSplit() in order to keep the chronological order of the data
cv = TimeSeriesSplit()
model = RandomizedSearchCV(DecisionTreeRegressor(),  
                     param_grid, cv=cv, n_jobs=-1, verbose=1, scoring='neg_root_mean_squared_error',
                     n_iter=50, random_state=4375) 
# We will also train the model without a scaler
model2 = GridSearchCV(DecisionTreeRegressor(),  
                     param_grid, cv=cv, n_jobs=-1, verbose=1, scoring='neg_root_mean_squared_error')
# Model without HPO
model_whpo = GridSearchCV(DecisionTreeRegressor(),
                          {}, cv=cv, n_jobs=-1, verbose=1, scoring='neg_root_mean_squared_error')

pipe = Pipeline([
    ('scaler', PowerTransformer()),
    ('model', model)
])

pipe_whpo = Pipeline([
    ('scaler', PowerTransformer()),
    ('model', model_whpo)
])

a = time.time()
pipe.fit(X_train, y_train)
b = time.time()
model2.fit(X_train, y_train)
c = time.time()
pipe_whpo.fit(X_train, y_train)
d = time.time()

best_params = pipe.named_steps['model'].best_params_
score = pipe.named_steps['model'].best_score_

best_params_whpo = pipe_whpo.named_steps['model'].best_params_
score_whpo = pipe_whpo.named_steps['model'].best_score_

print("With scaler <-----> Without scaler <-----> Without HPO")
print(f"Best params: {best_params} <------> {model2.best_params_} <------> {best_params_whpo}")
print(f"Best score: {-score} <------> {model2.best_score_} <------> {-score_whpo}")
print(f"Time taken to train the DecisionTreeRegressor Model: {b-a} seconds <-----> {c-b} seconds <------> {d-c} seconds")

graph_data["Decision_Tree_R"] = [-score, b-a]

# Resultado con data sin filtrar
# Fitting 5 folds for each of 50 candidates, totalling 250 fits
# Best params: {'min_samples_split': 2, 'min_samples_leaf': 18, 'max_depth': 12, 'criterion': 'squared_error'}
# Best score: 425.7363157160712
# Time taken to train the DecisionTreeRegressor Model: 19.370752096176147

# Resultado con data filtrada
# Fitting 5 folds for each of 50 candidates, totalling 250 fits
# Best params: {'min_samples_split': 8, 'min_samples_leaf': 16, 'max_depth': 15, 'criterion': 'poisson'}
# Best score: 426.0749548459118
# Time taken to train the DecisionTreeRegressor Model: 10.071922779083252 seconds

Como hemos podido observar, el modelo es bastante mejor que el dummy, aproximadamente, un tercio mejor. No obstante probaremos con más modelos para contrastar.

El próximo modelo es la **regresión lineal** normal

In [ ]:
param_grid = {
    'fit_intercept': [True, False],  # Whether to calculate the intercept for this model
    'copy_X': [True, False],  # Whether to make a copy of X or overwrite it
    'n_jobs': [-1],  # Number of jobs to run in parallel, -1 indicates using all processors
    'positive': [True, False]  # When set to True, forces the coefficients to be positive
}

# Crear el modelo de regresión lineal
cv = TimeSeriesSplit()

grid_search_model = GridSearchCV(LinearRegression(),
                           param_grid,
                           cv=cv,
                           scoring="neg_root_mean_squared_error"
                           )
without_hpo = GridSearchCV(LinearRegression(),
                           {},
                           cv=cv,
                           scoring="neg_root_mean_squared_error"
                           )

pipe = Pipeline([
    ('scaler', scaler),
    ('model', grid_search_model)
])

pipe_whpo = Pipeline([
    ('scaler', PowerTransformer()),
    ('model', without_hpo)
])

# Entrenar el modelo con los datos de entrenamiento
a = time.time()
pipe.fit(X_train, y_train)
b = time.time()
pipe_whpo.fit(X_train, y_train)
c = time.time()

best_params = pipe.named_steps['model'].best_params_
score = pipe.named_steps['model'].best_score_

best_params_whpo = pipe_whpo.named_steps['model'].best_params_
score_whpo = pipe_whpo.named_steps['model'].best_score_

print("With scaler <-----> Without HPO")
print(f"Best params: {best_params} <------> {best_params_whpo}")
print(f"Best score: {-score} <------> {-score_whpo}")
print(f"Time taken to train the DecisionTreeRegressor Model: {b-a} seconds <-----> {c-b} seconds")

graph_data["Linear_Regression"] = [-pipe.named_steps['model'].best_score_, b-a]


"""
best model: LinearRegression(n_jobs=-1)
best params: {'copy_X': True, 'fit_intercept': True, 'n_jobs': -1, 'positive': False}
RMSE:  549.6089612265775
Time taken to train the SVR: 0.3889331817626953 seconds
"""

Como podemos observar, aunque es mejor que el dummy, está bastante peor que el árbol de decisión, no obstante, vamos a probar con  con la variante **Lasso** a ver si conseguimos mejorar el entrenamiento:

In [ ]:
# Definir los parámetros que deseas probar
parameters = {
    'alpha': [0.1, 0.5, 1.0, 2.0],  # Regularization parameter
    'fit_intercept': [True, False],  # Calculate the intercept or not
    'copy_X': [True, False],  
    'positive': [True, False],  # When set to True, forces the coefficients to be positive
    'precompute': [True, False],  # Compute the Gram matrix or not
    'selection': ['cyclic', 'random'],  # Ways of fitting the model
    'tol': [1**(-10), 1**(-5), 1**(-4), 1**(-3), 1**(-2)],  # Tolerance for stopping criteria
    'warm_start': [True, False]  
}


# Crear el modelo de regresión lineal Lasso
cv = TimeSeriesSplit()
grid_search = GridSearchCV(Lasso(), 
                           parameters, 
                           scoring='neg_root_mean_squared_error', 
                           cv=cv)

without_hpo = GridSearchCV(Lasso(),
                           {},
                           cv=cv,
                           scoring="neg_root_mean_squared_error"
                           )

pipe = Pipeline([
    ('scaler', scaler),
    ('model', grid_search_model)
])

pipe_whpo = Pipeline([
    ('scaler', PowerTransformer()),
    ('model', without_hpo)
])

# Entrenar el modelo con los datos de entrenamiento
a = time.time()
pipe.fit(X_train, y_train)
b = time.time()
pipe_whpo.fit(X_train, y_train)
c = time.time()

best_params = pipe.named_steps['model'].best_params_
score = pipe.named_steps['model'].best_score_

best_params_whpo = pipe_whpo.named_steps['model'].best_params_
score_whpo = pipe_whpo.named_steps['model'].best_score_

print("With scaler <-----> Without HPO")
print(f"Best params: {best_params} <------> {best_params_whpo}")
print(f"Best score: {-score} <------> {-score_whpo}")
print(f"Time taken to train the DecisionTreeRegressor Model: {b-a} seconds <-----> {c-b} seconds")


graph_data["Lasso"] = [-grid_search.best_score_, b-a]

"""
best model: Lasso(alpha=0.1, precompute=True, random_state=7543, selection='random',
      tol=1.0, warm_start=True)
best params: {'alpha': 0.1, 'copy_X': True, 'fit_intercept': True, 'positive': False, 'precompute': True, 'random_state': 7543, 'selection': 'random', 'tol': 1.0, 'warm_start': True}
RMSE:  566.6685051282924
Time taken to train the SVR: 27.60527014732361 seconds
"""

Desafortunadamente, el modelo es ligeramente peor que el de  regresión lineal normal, pero como los árboles le sacan bastante más precisión (aunque tardan más en entrenar), seguiremos decantándonos por ellos. 

No obstante, aun quedan algunos modelos por probar, por lo que el siguiente es el **KNN**

In [ ]:
# Usaremos los parametros por default n_slits = 5, max_train_dize = None, test_size = None, gap = 0 
cv = TimeSeriesSplit()

param_grid = {
    'n_neighbors': range(5, 40, 5),  # Number of neighbors to consider
    'weights' : ['uniform', 'distance'],  # Weight function used in prediction
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],  # Algorithm used to compute nearest neighbors
    'leaf_size': range(1, 25, 5),  # Leaf size passed to BallTree or KDTree
    'p' : [1,2],  # Distance metric (1 for Manhattan, 2 for Euclidean)
}


grid_search = GridSearchCV(KNeighborsRegressor(),
                     param_grid, cv=cv, n_jobs=-1, scoring='neg_root_mean_squared_error')

without_hpo = GridSearchCV(KNeighborsRegressor(),
                           {}, cv=cv, n_jobs=-1, scoring='neg_root_mean_squared_error')


pipe = Pipeline([
    ('scaler', PowerTransformer()),
    ('model', grid_search)
])

pipe_whpo = Pipeline([
    ('scaler', PowerTransformer()),
    ('model', without_hpo)
])


grid_search2 = GridSearchCV(KNeighborsRegressor(),
                     param_grid, cv=cv, n_jobs=-1, scoring='neg_root_mean_squared_error')

a = time.time()
pipe.fit(X_train, y_train)
b = time.time() 
grid_search2.fit(X_train, y_train)
c = time.time()
pipe_whpo.fit(X_train, y_train)
d = time.time()

best_model = grid_search2.best_estimator_
best_params = grid_search2.best_params_
best_score = -grid_search2.best_score_

best_model_pipe = pipe.named_steps["model"].best_estimator_
best_params_pipe = pipe.named_steps["model"].best_params_
best_score_pipe = -pipe.named_steps["model"].best_score_

best_model_whpo = pipe_whpo.named_steps['model'].best_estimator_
best_params_whpo = pipe_whpo.named_steps['model'].best_params_
score_whpo = pipe_whpo.named_steps['model'].best_score_

print("With scaler <-----> Without scaler <-----> Without HPO")
print(f"Best model: {best_model_pipe} <------> {best_model} <------> {best_model_whpo}")
print(f"Best params: {best_params_pipe} <------> {best_params} <------> {best_params_whpo}")
print(f"Best score: {best_score_pipe} <------> {best_score} <------> {score_whpo}")
print(f"Time taken to train the DecisionTreeRegressor Model: {b-a} seconds <-----> {c-b} seconds <------> {d-c} seconds")

graph_data["KNN"] = [best_score_pipe, b-a]


# Resultado con data sin filtrar
# Mejor modelo: KNeighborsRegressor(leaf_size=1, n_neighbors=35, p=1) <-----------> KNeighborsRegressor(leaf_size=1, n_neighbors=15, p=1, weights='distance')
# Mejorres parámetros: {'algorithm': 'auto', 'leaf_size': 1, 'n_neighbors': 35, 'p': 1, 'weights': 'uniform'} <-----------> {'algorithm': 'auto', 'leaf_size': 1, 'n_neighbors': 15, 'p': 1, 'weights': 'distance'}
# Score: 593.643591243938 <-----------> 412.17437045226825
# Time taken to train the KNeighborsRegressor Model with scaler: 30.788731813430786 seconds
# Time taken to train the KNeighborsRegressor Model without scaler: 12.302868366241455 seconds

# Resultado con data filtrada
# Mejor modelo: KNeighborsRegressor(leaf_size=1, n_neighbors=35, p=1) <-----------> KNeighborsRegressor(leaf_size=1, n_neighbors=15, p=1, weights='distance')
# Mejorres parámetros: {'algorithm': 'auto', 'leaf_size': 1, 'n_neighbors': 35, 'p': 1, 'weights': 'uniform'} <-----------> {'algorithm': 'auto', 'leaf_size': 1, 'n_neighbors': 15, 'p': 1, 'weights': 'distance'}
# Score: 593.643591243938 <-----------> 412.17437045226825
# Time taken to train the KNeighborsRegressor Model with scaler: 12.555383205413818 seconds
# Time taken to train the KNeighborsRegressor Model without scaler: 5.734422922134399 seconds

Como hemos podido observar, el uso del scaler cambia totalmente el resultado obtenido, convirtiendose así en el mejor modelo hasta la fecha y el más probable para el entrenamiendo final, pero, antes de apresurarnos, probaremos con **SVR**


In [ ]:
cv = TimeSeriesSplit()

param_grid = {
    'C': range(705, 720),  # Regularization parameter
    'epsilon': [0.1, 0.2, 0.3], # Controls the width of the margin around the regression line within which no penalty is incurred.
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],  # Kernel type
    'degree': [1, 2, 3],  # Degree of the polynomial kernel (only for 'poly' kernel)
    'gamma': ['scale', 'auto']  # Kernel coefficient for 'rbf', 'poly', and 'sigmoid'
}

grid_search = GridSearchCV(SVR(),
                            param_grid,
                            cv=cv, 
                            n_jobs=-1, 
                            scoring='neg_root_mean_squared_error')

without_hpo = GridSearchCV(SVR(),
                           {}, cv=cv, n_jobs=-1, scoring='neg_root_mean_squared_error')



pipe = Pipeline([
    ('scaler', PowerTransformer()),
    ('model', grid_search)
])

pipe_whpo = Pipeline([
    ('scaler', PowerTransformer()),
    ('model', without_hpo)
])


a = time.time()
pipe.fit(X_train, y_train)
pipe_whpo.fit(X_train, y_train)
c = time.time()

best_params = pipe.named_steps['model'].best_params_
score = pipe.named_steps['model'].best_score_

best_params_whpo = pipe_whpo.named_steps['model'].best_params_
score_whpo = pipe_whpo.named_steps['model'].best_score_

print("With scaler <-----> Without HPO")
print(f"Best params: {best_params} <------> {best_params_whpo}")
print(f"Best score: {-score} <------> {-score_whpo}")
print(f"Time taken to train the DecisionTreeRegressor Model: {b-a} seconds <-----> {c-b} seconds")


graph_data["KNN"] = [-pipe.named_steps['model'].best_score_, b-a]


# time taken to run the code:
# time = "13m 46s"
# 384.2528063686719
# best model: SVR(C=719, degree=1, gamma='auto')
# best params: {'C': 719, 'degree': 1, 'epsilon': 0.1, 'gamma': 'auto', 'kernel': 'rbf'}
# RMSE:  380.9578481516544

# Resultado con data sin filtrar
# 447.31691802522346
# best model: SVR(C=719, degree=1, gamma='auto')
# best params: {'C': 719, 'degree': 1, 'epsilon': 0.1, 'gamma': 'auto', 'kernel': 'rbf'}
# RMSE:  380.9578440914255
# Time taken to train the SVR: 266.7573239803314 seconds ~ 4.4 minutes

# Resultado con data filtrada
# 434.6028671711526
# best model: SVR(C=719, degree=1, gamma='auto')
# best params: {'C': 719, 'degree': 1, 'epsilon': 0.1, 'gamma': 'auto', 'kernel': 'rbf'}
# RMSE:  380.9578440914255
# Time taken to train the SVR: 284.6029191017151 seconds

Y como podemos observar, los mejores modelos son SVR, KNN con la pipe creada y el Decision Tree regresor.

A pesar de que el KNN se entrena en menos tiempo y tiene las segundas mejores métricas, usaremos el SVR para la versión final ya que consideramos que la diferencia de tiempo no es tan relevante como el beneficio que se obtiene. Y siguiendo las reglas previamente mencionadas, el tiempo nos importa un 20% y la precisión un 80%, por lo que 
(266)*0.2 + (380)*0.8 = 357.2
(30)*0.2 + (412)*0.8 = 335.6

Antes de entrenar el modelo final, vamos a hacer una comparación de todos los modelos hasta la fecha, tenemos dos diccionarios, una que va a estar "hardcodeada" para evitar tener que ejecutar todos los modelos, y otra que va a ser la que se guarde después de la ejecución de todos los modelos, usaremos las listas para mostrar una grafica comparando los tiempos y la precisión.

Cabe destacar que las medidas de tiempo no son muy precisas ya que el número de hiperparámetros que probamos en diferentes modelos varía, y por lo tanto es normal que tarden más.

In [ ]:
# Datos de resultados
results = {"Dummy": [663, 0.087], "Decision_Tree_R": [425, 19.37], "Linear_Regression": [549, 0.38], "Lasso": [566, 27.60], "KNN": [412, 30.78], "SVR": [380, 266.75]}

#### Graficas generadas por ChatGPT ####

# Extraer nombres de modelos y puntajes
modelos = list(results.keys())
scores = [score[0] for score in results.values()]
tiempos = [tiempo[1] for tiempo in results.values()]

# Crear el gráfico de barras
plt.figure(figsize=(12, 8))

# Barra para los puntajes (RMSE)
plt.barh(modelos, scores, color='skyblue', label='RMSE')

# Barra para los tiempos de entrenamiento
plt.barh(modelos, tiempos, color='salmon', alpha=0.7, label='Tiempo de entrenamiento (seg)')

# Etiquetas y título
plt.xlabel('Score / Tiempo de entrenamiento')
plt.ylabel('Modelo')
plt.title('Comparación de Puntajes y Tiempos de Entrenamiento')
plt.legend()

# Mostrar gráfico
plt.gca().invert_yaxis()  # Invertir el eje y para mostrar el mejor modelo arriba
plt.grid(axis='x')  # Añadir rejilla solo en el eje x para mayor claridad
plt.show()


Y por último, vamos a preparar el modelo final para la competición:

In [ ]:
"""
Al final el modelo seleccionado es el SVR, la variante para problemas de regresión de SVM, este tipo de modelos se basan en maquinas de soporte vectorial cuyo objetivo es minimizar la suma de la diferencia entre las predicciones y los valores reales.
"""
pipe = Pipeline([
    ('scaler', PowerTransformer()),
    ('model', SVR(C=719, degree=1, epsilon=0.1, gamma='auto', kernel='rbf'))
])


pipe.fit(X_train, y_train)

y_pred_pipe = pipe.predict(X_test)

rmse_pipe = np.sqrt(mean_squared_error(y_test, y_pred_pipe))
print("RMSE:", rmse_pipe)

final_pipe = Pipeline([
    ('scaler', PowerTransformer()),
    ('model', SVR(C=719, degree=1, epsilon=0.1, gamma='auto', kernel='rbf'))
])

final_pipe.fit(X, y)

# Save the model to a file
dump(final_pipe, 'best_model_svr.joblib')

# Uso de ChatGPT en esta parte:


Como bien está comentado en el código, ChatGPT se usó para generar la gráfica que muestra los tiempos de ejecución y las scorings de todos los modelos.
